In [1]:
from itertools import combinations

def all_valuations(variables):
    for k in range(len(variables)+1):
        for true_variables in combinations(variables, k):
            valuation = {x: False for x in variables}
            valuation.update({x: True for x in true_variables})
            yield valuation

class Formula:
    def __init__(self):
        pass

    def interpret(self, valuation):
        pass

    def __eq__(self, r):
        return Eq(self, r)

    def __and__(self, r):
        return And(self, r)

    def __or__(self, r):
        return Or(self, r)
    
    def __invert__(self):
        return Not(self)

    def __rshift__(self, r):
        return Impl(self, r)
    

    def get_all_variables(self):
        result = set()
        for c in self.components:
            result.update(c.get_all_variables())
        return result

    def is_valid(self):
        for valuation in all_valuations(self.get_all_variables()):
            if self.interpret(valuation) == False:
                return False, valuation
        return True, None
    
    def is_satisfiable(self):
        for valuation in all_valuations(self.get_all_variables()):
            if self.interpret(valuation) == True:
                return True, valuation
        return False, None
    
    def is_falsifiable(self):
        for valuation in all_valuations(self.get_all_variables()):
            if self.interpret(valuation) == False:
                return True, valuation
        return False, None
    
    def is_contradictory(self):
        for valuation in all_valuations(self.get_all_variables()):
            if self.interpret(valuation) == True:
                return False, valuation
        return True, None


class Const(Formula):
    def __init__(self, value):
        super().__init__()
        self.value = value

    def interpret(self, valuation):
        return self.value

    def get_all_variables(self):
        return set()

    def __str__(self):
        return '1' if self.value else '0'

class Var(Formula):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def interpret(self, valuation):
        return valuation[self.name]

    def get_all_variables(self):
        return set([self.name])

    def __str__(self):
        return self.name



class And(Formula):
    def __init__(self, l, r):
        super().__init__()
        self.components = [l, r]

    def interpret(self, valuation):
        return self.components[0].interpret(valuation) and self.components[1].interpret(valuation)

    def __str__(self):
        return f'({self.components[0]}) & ({self.components[1]})'

class Or(Formula):
    def __init__(self, l, r):
        super().__init__()
        self.components = [l, r]

    def interpret(self, valuation):
        return self.components[0].interpret(valuation) or self.components[1].interpret(valuation)

    def __str__(self):
        return f'({self.components[0]}) | ({self.components[1]})'

class Eq(Formula):
    def __init__(self, l, r):
        super().__init__()
        self.components = [l, r]

    def interpret(self, valuation):
        return self.components[0].interpret(valuation) == self.components[1].interpret(valuation)

    def __str__(self):
        return f'({self.components[0]}) == ({self.components[1]})'

class Impl(Formula):
    def __init__(self, l, r):
        super().__init__()
        self.components = [l, r]

    def interpret(self, valuation):
        return not self.components[0].interpret(valuation) or self.components[1].interpret(valuation)

    def __str__(self):
        return f'({self.components[0]}) >> ({self.components[1]})'

class Not(Formula):
    def __init__(self, op):
        super().__init__()
        self.components = [op]

    def interpret(self, valuation):
        return not self.components[0].interpret(valuation)

    def __str__(self):
        return f'~({self.components[0]})'


def evaluate_formula(formula):
    print('Valid: ', formula.is_valid())
    print('Satisfiable: ', formula.is_satisfiable())
    print('Falsifiable: ', formula.is_falsifiable())
    print('Contradictory: ', formula.is_contradictory())

    print('\nAll valid configurations:')
    for valuation in all_valuations(formula.get_all_variables()):
        if formula.interpret(valuation):
            print(valuation)


def vars(names):
    return [Var(name.strip()) for name in names.split(',')]


***

## Examples

In [2]:
'''
Three fields are colored red or blue.
If the first one is red, the other two must be the same color.
If the second is red, the third must be blue.
'''
A,B,C = vars("A,B,C") # A:True => A red
formula = (A >> (B == C)) & (B >> ~C)
evaluate_formula(formula)

Valid:  (False, {'C': True, 'A': True, 'B': False})
Satisfiable:  (True, {'C': False, 'A': False, 'B': False})
Falsifiable:  (True, {'C': True, 'A': True, 'B': False})
Contradictory:  (False, {'C': False, 'A': False, 'B': False})

All valid configurations:
{'C': False, 'A': False, 'B': False}
{'C': True, 'A': False, 'B': False}
{'C': False, 'A': True, 'B': False}
{'C': False, 'A': False, 'B': True}


In [3]:
'''
Write that the 4-bit representation of the number is a palindrome,
but bits are not equal ABCD
'''

A,B,C,D = vars("A,B,C,D")
formula = (A == D) & (B == C) & ~((A == B) & (B == C) & (C == D)) # ! Brackets because of priority!
evaluate_formula(formula)

Valid:  (False, {'D': False, 'C': False, 'A': False, 'B': False})
Satisfiable:  (True, {'D': True, 'C': False, 'A': True, 'B': False})
Falsifiable:  (True, {'D': False, 'C': False, 'A': False, 'B': False})
Contradictory:  (False, {'D': True, 'C': False, 'A': True, 'B': False})

All valid configurations:
{'D': True, 'C': False, 'A': True, 'B': False}
{'D': False, 'C': True, 'A': False, 'B': True}


In [4]:
'''
The following configuration was obtained in the 2x3 minesweeper game
|1|A|C|
|1|B|2|
A, B, C are unopened fields, and the numbers indicate the number of mines in the surrounding fields.
'''
A,B,C = vars("A,B,C")
formula = (A | B) & ~(A & B) \
    & (B | A) & ~(B & A) \
    & ~(~A & ~B & ~C)\
    & (A | B)\
    & (B | C)\
    & (A | C)\
    & ~(A & B & C)
evaluate_formula(formula)

Valid:  (False, {'C': False, 'A': False, 'B': False})
Satisfiable:  (True, {'C': True, 'A': True, 'B': False})
Falsifiable:  (True, {'C': False, 'A': False, 'B': False})
Contradictory:  (False, {'C': True, 'A': True, 'B': False})

All valid configurations:
{'C': True, 'A': True, 'B': False}
{'C': True, 'A': False, 'B': True}
